# Music Generation - Siddharth Shah, Ian Pompliano

Artificial Neural Networks and Deep Learning Final Project (Fall 2023)

**Part 1: Scrape midi files.**

In [1]:
import requests
import os
from bs4 import BeautifulSoup
from music21 import *

In [2]:
# list of composers
composers = ["albeniz", "bach", "balak", "beeth", "borodin", "brahms", "burgm",
             "chopin", "clementi", "debuss", "godowsky", "grana", "grieg", "haydn",
             "liszt", "mendelssohn", "moszkowski", "mozart", "muss", "rach", "ravel",
             "schub", "schum", "sinding", "tschai"]

baseURL = 'http://www.piano-midi.de/'

# create a directory to save downloaded MIDI files
if not os.path.exists('midiFiles'):
    os.makedirs('midiFiles')

for composer in composers:
    # create URL using respective composer
    url = baseURL + composer + '.htm'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find MIDI links on the composer's page
    midiLinks = soup.find_all('a', href=True)

    for link in midiLinks:
        # removes duplicate midi files (special format0)
        if link['href'].endswith('.mid') and '_format0' not in link['href']:
            midiURL = baseURL + link['href']
            fileName = f"midiFiles/{composer}_{link['href'].split('/')[-1]}"

            # download midi file and save it to directory
            with open(fileName, 'wb') as midiFile:
                midiResponse = requests.get(midiURL)
                midiFile.write(midiResponse.content)

**Part 2: Organize notes in each MIDI file. Create list of sublists containing notes for each song**

In [3]:
# helper function to get the notes in a given MIDI file
def getNotes(file):
    # initalize list of notes to return
    notes = []
    pick = file.recurse()

    for element in pick:
        # if element is note, add to list of notes
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        # if element is chord, add highest pitch (generally belongs to melody) to list of notes.
        elif isinstance(element, chord.Chord):
            highestPitch = max(element.pitches)
            notes.append(str(highestPitch))
    return notes

In [4]:
# initialize list that will hold sub-lists of notes for each song
allNotes = []

# retrieve paths of MIDI files
midiFiles = [os.path.join('midiFiles', file) for file in os.listdir('midiFiles') if file.endswith('.mid')]

"""
for path in midiFiles:
    midi = converter.parse(path)
    notes = getNotes(midi)
    allNotes.append(notes)
"""
midi = converter.parse(midiFiles[6])
notes = getNotes(midi)

allNotes.append(notes)

print("Notes for the first MIDI file:")
print(allNotes[0])


/Users/ianpompliano/anaconda3/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2007 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


Notes for the first MIDI file:
['G5', 'B4', 'C5', 'D5', 'D5', 'D5', 'C5', 'D5', 'E-5', 'D5', 'C5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'D5', 'E-5', 'D5', 'B4', 'C5', 'D5', 'D5', 'D5', 'C5', 'D5', 'E-5', 'D5', 'C5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'E-5', 'D5', 'C5', 'D5', 'E-5', 'F5', 'F5', 'F5', 'E-5', 'D5', 'G5', 'F5', 'E-5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'F5', 'E-5', 'D5', 'D5', 'E-5', 'F5', 'F5', 'F5', 'E-5', 'D5', 'G5', 'F5', 'E-5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'E-5', 'D5', 'C5', 'B4', 'C5', 'D5', 'D5', 'D5', 'C5', 'D5', 'D5', 'E-5', 'D5', 'C5', 'D5', 'C5', 'A4', 'B4', 'A4', 'B4', 'C5', 'D5', 'E-5', 'D5', 'B4', 'C5', 'D5', 'D5', 'D5', 'C5', 'D5', 'E-5', 'D5', 'C5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'E-5', 'D5', 'C5', 'D5', 'E-5', 'F5', 'F5', 'F5', 'B-4', 'F5', 'E-5', 'D5', 'B-4', 'D5', 'E-5', 'D5', 'C5', 'G5', 'F5', 'B4', 'A4', 'B4', 'C5', 'F5', 'E-5', 'D5', 'D5', 'E-5', 'F5', 'F5', 'F5', 'E-5', 'D5', 'G5', 'F5', 'E-5', 'D5', 'C5', 'B4', 'A4', 'B4', 'C5', 'E-5', 

In [5]:
notes = allNotes[0]
note_stream = stream.Stream()

for n in notes:
    noteObj = note.Note(n)
    note_stream.append(noteObj)

note_stream.append(tempo.MetronomeMark(number=120))
midi_file = note_stream.write('midi', fp='test.mid')